In [ ]:
# Install py3Dmol using pip
!pip install py3Dmol
# Install biopython using pip
!pip install biopython
# Install pdb2pqr v3.0 using pip
!pip install pdb2pqr
# We will also install rdkit
!pip install rdkit
# Install vina usiung pip
!pip install vina

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 19.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 207 kB 28.1 MB/s 
     |████████████████████████████████| 98 kB 9.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.3 MB 101.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 22.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 22.9 MB/s 


In [ ]:
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Solving environment: - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mgltools
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    cairo-1.16.0               |       h19f5f5c_2         1.1 MB
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-4.14.0               |   py38h578d9bd_0        1012 K

In [ ]:
import os
os.cpu_count()

8

In [ ]:
#Downloading the PDB files using biopython
import os
from Bio.PDB import *
pdbid = ['1t5c']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path 

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
crosspath = Path("/content/cross-dock/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(crosspath):
  print("path already exists")
if not os.path.exists(crosspath):
  os.mkdir(crosspath)
  print("path was succesfully created")

#Now we assign a variable "protein" with the name and extension of our pdb
protein = "1t5c.pdb"

#And we use the following script to selectively print the lines that contain the
#string "ATOM" and "TER" into a new file inside our recently created folder
with open(crosspath / "1t5c_prot.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM": # and row[4] == "A":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

path was succesfully created
file successfully created


In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#We will first create a path for all ligands that we will use in this tutorial
#Notice that the HOME folder for a hosted runtime in colab is /content/
ligandpath = Path("/content/ligands/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(ligandpath):
  print("ligand path already exists")
if not os.path.exists(ligandpath):
  os.mkdir(ligandpath)
  print("ligand path was succesfully created")

ligand path was succesfully created


In [ ]:
#Downloading NExT 3500 diversity library from uploaded file in sdf format
from google.colab import drive
drive.mount('/content/drive')

# Test code
# !cp 'drive/MyDrive/test.sdf' $ligandpath

!cp 'drive/MyDrive/multiprocessing_test.sdf' $ligandpath

Mounted at /content/drive
/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)


In [ ]:
# Prepare the protein and ligands
import subprocess

# subprocess.run(['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pdb', 
#                 '-o', '/content/cross-dock/1t5c_prot.pdbqt', 
#                 '-A', 'hydrogens', '-U', 'nphs_lps', '-v'])    # causing unknown parsing error
# subprocess.call(f'prepare_receptor4.py -r /content/cross-dock/1t5c_prot.pdb 
#                   -o /content/cross-dock/1t5c_prot.pdbqt -A hydrogens -U nphs_lps -v', 
#                 shell = True)    # old version
subprocess.run(['pdb2pqr30', '--ff', 'AMBER', '--keep-chain', 
                '--titration-state-method', 
                'propka', '--with-ph', '7.4', 
                '/content/cross-dock/1t5c_prot.pdb', 
                '/content/cross-dock/1t5c_prot.pqr'])
subprocess.run(['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pqr', 
                '-o', '/content/cross-dock/1t5c_prot.pdbqt', '-C', '-U', 'nphs_lps', '-v'])

CompletedProcess(args=['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pqr', '-o', '/content/cross-dock/1t5c_prot.pdbqt', '-C', '-U', 'nphs_lps', '-v'], returncode=0)

In [ ]:
from rdkit import Chem

# Import sdf file
sdf_path = "/content/ligands/multiprocessing_test.sdf"
# sdf_path = "/content/ligands/Diversity_3500_Library_modified.sdf"

# Read sdf by rdkit
suppl = Chem.SDMolSupplier(sdf_path, removeHs = False)
mols_info = []
for mol in suppl:
  mol_id = mol.GetProp('COMPOUND_ID')
  smi = mol.GetProp('SMILES')
  mols_info.append([mol, mol_id, smi])

In [ ]:
from vina import Vina
from rdkit.Chem.rdMolTransforms import ComputeCentroid

s = open("/content/scores.txt", "w")

# Cross docking function
def docking(mol_info):

  # Get process info
  curr_proc = multiprocessing.current_process()
  proc_id = curr_proc.name

  s = open("/content/scores.txt", "a")

  # Acquire mol data and compute centroid of ligand compound
  mol = mol_info[0]
  centroid = ComputeCentroid(mol.GetConformer())

  # Acquire compound id and smiles string
  mol_id = mol_info[1]
  smi = mol_info[2]

  # Write smiles file
  with open(ligandpath / f'ligand{proc_id}.smi', 'w') as f:
    f.write(smi)

  # Prepare ligand file from .smi to .pdbqt
  # three different methods
  # a) Use the program babel to convert the SMILES into a MOL2 file without any extra work 
  # (such as searching for best conformers) except for setting the protonation state to pH 7.4, 
  # and then use MGLtools to parameterize the ligand using Gasteiger partial charges
  subprocess.run(['obabel', f'/content/ligands/ligand{proc_id}.smi', 
                  '-O', f'ligand{proc_id}.mol2', '--gen3d', 'best', '-p', '7.4', '--canonical'])
  subprocess.run(['prepare_ligand4.py', '-l', f'ligand{proc_id}.mol2', '-o', 
                  f'/content/ligands/ligand{proc_id}.pdbqt', '-U', 'nphs_lps', '-v'])
  os.remove(f'ligand{proc_id}.mol2')

  # Setup AutoDock Vina
  v = Vina(sf_name = 'vina')
  v.set_receptor('/content/cross-dock/1t5c_prot.pdbqt')
  v.set_ligand_from_file(f'/content/ligands/ligand{proc_id}.pdbqt')
  v.compute_vina_maps(center = [centroid.x, centroid.y, centroid.z], 
                          box_size = [30, 30, 30])

  # Dock the ligand
  v.dock(exhaustiveness = 24, n_poses = 9)
  dock_energy = v.score()
  print('Score after docking : %.3f (kcal/mol)' % dock_energy[0])
  # v.write_poses(f'1t5c_ligand{id}_vina_out.pdbqt', n_poses = 9, overwrite = True)

  # Write compound id, smiles string, and score in score file
  s.write(str(mol_id) + ' ' + smi + ' ' + '%.3f\n' % dock_energy[0])

  # Remove .smi and .pdbqt files
  os.remove(f'/content/ligands/ligand{proc_id}.smi')
  os.remove(f'/content/ligands/ligand{proc_id}.pdbqt')

  # Close the file
  s.close()

In [ ]:
import multiprocessing
with multiprocessing.Pool(processes = 8) as pool:
	# call the function for each item in parallel
	pool.map(docking, mols_info)

Score after docking : -6.330 (kcal/mol)
Score after docking : -5.932 (kcal/mol)
Score after docking : -7.641 (kcal/mol)
Score after docking : -6.202 (kcal/mol)
Score after docking : -6.512 (kcal/mol)
Score after docking : -7.066 (kcal/mol)
Score after docking : -6.995 (kcal/mol)
Score after docking : -7.230 (kcal/mol)
Score after docking : -6.156 (kcal/mol)
Score after docking : -7.547 (kcal/mol)
Score after docking : -7.467 (kcal/mol)
Score after docking : -6.811 (kcal/mol)
Score after docking : -6.845 (kcal/mol)
Score after docking : -6.193 (kcal/mol)
